# TEST loading RTs to Sinopia
- @mcm104 wrote [s2s04json2sinopia.py](s2s04json2sinopia.py)
- I'm trying to understand it and make it work
- Going to attempt to load [../docs/rdf/TEST_WAU_rdacManifestation_test_literal_props_ries07.json](../docs/rdf/TEST_WAU_rdacManifestation_test_literal_props_ries07.json)


## But first, a digression
Need to understand the `str.split()` method a bit better

In [ ]:
string = "well.now.here's.a.test"
print(string.split(".")[0])

In [ ]:
print(f"{string.split('.')[0]} - {string.split('.')[2]} - {string.split('.')[4]}")

In [ ]:
"""
import os
import requests

jwt = input("Enter the jwt:\n")
headers = {"Authorization": f"Bearer {jwt}", "Content-Type": "application/json"}
"""

file = "../docs/rdf/TEST_WAU_rdacManifestation_test_literal_props_ries07.json"
open_file = open(file)
data = open_file.read()

# what is 'file'? filename?
print(file)

# create a file id
file_id = file.split("rdf/")[1]
print(file_id)
file_id = file_id.split(".")[0]
print(file_id)
file_id = file_id.replace("_", ":")
print(file_id)

## Attempt post...

In [ ]:
import os
import requests

environment = input("Enter the Sinopia environment to post to ('production', 'stage', or 'development'):\n")
jwt = input("Enter the jwt:\n")
headers = {"Authorization": f"Bearer {jwt}", "Content-Type": "application/json"}
file = "../docs/rdf/TEST_WAU_rdacManifestation_test_literal_props_ries07.json"

# create a file uri
file_id = file.split("rdf/")[1]
file_id = file_id.split(".")[0]
file_id = file_id.replace("_", ":")
file_uri = f"https://api.{environment}.sinopia.io/resource/{file_id}"
print(file_uri)

# I still don't understand everything from here down all that well

# post
post_to_sinopia = requests.post(file_uri, data=data.encode('utf-8'), headers = headers)

# check for errors
if post_to_sinopia.status_code != requests.codes.ok:
    error_code = post_to_sinopia.status_code
    if error_code == 201: # created, not actually an error
        print("Success!")
    elif error_code == 401: # unauthorized
        print("\nFailed: Java Web Token no longer valid.")
    elif error_code == 409: # conflict
        print("\nFailed: IRI is not unique. Putting instead...")
        post_to_sinopia_2 = requests.put(file_uri, data=data.encode('utf-8'), headers = headers)
        if post_to_sinopia_2.status_code != requests.codes.ok:
            error_code = post_to_sinopia_2.status_code
            if error_code != 201:
                print(f"Failed: {error_code}")
            else:
                print("Success!")
        else:
            print("Success!")
    else:
        print(f"Failed: {error_code}")
else:
    print("Success!")

### ...and notes/questions
NOTES/QUESTIONS
- RT has `"user": "ries07uwdev"`, and I'm logged in as `ries07uwdev` when I retrieve the JWT. Is this alignment required?
- Still failing with a `400`
- Most likely need to read [Sinopia API ref](https://ld4p.github.io/sinopia_api/)
- Note also that the selection of environment (production, stage, or development) is actually made at an earlier stage in the Python-processing, but that (*I believe*) the environment selected at that time must be the same environment selected here
